In [32]:
import datalake
import importlib
import logging
import os
import pymongo
import stackoverflow

importlib.reload(stackoverflow)
importlib.reload(datalake)
TAG = 'data-science'


In [19]:

# Set up logging ... DEBUG for my code, WARNING for libraries
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("hdfs").setLevel(logging.WARNING)
logging.basicConfig(format='%(asctime)s - %(levelname)-6s - %(name)15s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)

mylake = datalake.DataLake('raw', 'www.stackoverflow.com', 'questions-by-tag')

so = stackoverflow.Scraper()

client = pymongo.MongoClient(host='mongodb')
db = client.stackoverflow
coll = db.questions_by_tag


In [20]:

class HtmlLakeWriter(object):

    def __init__(self):
        self.page = 1

    def on_result(self, result):
        mylake.store_html('tag-{tag}-{page:06d}'.format(tag=TAG, page=self.page),
                          result)
        self.page = self.page + 1

lakeWriter = HtmlLakeWriter()

so.search_tag(lakeWriter.on_result, TAG)


2021-10-02 01:36:32 - INFO   - stackoverflow.Scraper - Searching StackOverflow for questions with tag: data-science
2021-10-02 01:36:33 - DEBUG  - stackoverflow.Scraper - Retrieving page 2 of 140
2021-10-02 01:36:35 - DEBUG  - stackoverflow.Scraper - Retrieving page 3 of 140
2021-10-02 01:36:36 - DEBUG  - stackoverflow.Scraper - Retrieving page 4 of 140
2021-10-02 01:36:37 - DEBUG  - stackoverflow.Scraper - Retrieving page 5 of 140
2021-10-02 01:36:38 - DEBUG  - stackoverflow.Scraper - Retrieving page 6 of 140
2021-10-02 01:36:40 - DEBUG  - stackoverflow.Scraper - Retrieving page 7 of 140
2021-10-02 01:36:41 - DEBUG  - stackoverflow.Scraper - Retrieving page 8 of 140
2021-10-02 01:36:42 - DEBUG  - stackoverflow.Scraper - Retrieving page 9 of 140
2021-10-02 01:36:44 - DEBUG  - stackoverflow.Scraper - Retrieving page 10 of 140
2021-10-02 01:36:45 - DEBUG  - stackoverflow.Scraper - Retrieving page 11 of 140
2021-10-02 01:36:46 - DEBUG  - stackoverflow.Scraper - Retrieving page 12 of 140
2

In [21]:
mylake.list()

/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000001.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000002.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000003.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000004.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000005.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000006.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000007.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000008.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000009.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000010.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-000011.html
/raw/www.stackoverflow.com/questions-by-tag/2021/10/02/tag-data-science-0000

In [30]:
for path, dirs, files in mylake.walk():
    for f in files:
        content = mylake.get_html(os.path.join(path, f),
                                  stackoverflow.PageOfTaggedQuestions)

        content.iterate_questions(lambda question: coll.update_one({'qid': question['qid']},
                                                                   {'$set': question},
                                                                   upsert=True))
